In [1]:
import sys
sys.path.append('../../')

from dl.data.txtrecog import datasets, target_transforms, transforms
from dl.data.txtrecog.utils import batch_ind_fn

from dl.models.crnn import CRNN
from torch.nn import CTCLoss
from dl.optim.scheduler import IterStepLR
from dl.log import *

from torch.utils.data import DataLoader
from torch.optim.adadelta import Adadelta
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
augmentation = None

transform = transforms.Compose(
    [transforms.Resize((100, 32)),
     transforms.Grayscale(last_dims=1),
     transforms.ToTensor(),
     #transforms.Normalize(rgb_means=(0.485, 0.456, 0.406), rgb_stds=(0.229, 0.224, 0.225))
     # normalize 0.5, 0.5?: https://github.com/pytorch/vision/issues/288
     ]
)
target_transform = target_transforms.Compose(
    [target_transforms.Text2Number(class_labels=datasets.ALPHANUMERIC_WITH_BLANK_LABELS),
     target_transforms.ToTensor()
     ]
)

train_dataset = datasets.SynthTextRecognitionDataset(transform=transform, target_transform=target_transform, augmentation=augmentation)
train_loader = DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          collate_fn=batch_ind_fn,
                          num_workers=4,
                          pin_memory=True)

INFO:root:Loading ground truth...


KeyboardInterrupt: 

In [ ]:
model = CRNN(class_labels=datasets.ALPHANUMERIC_WITH_BLANK_LABELS, input_shape=(32, None, 1)).cuda()
model.train()
print(model)

In [ ]:
optimizer = Adadelta(model.parameters())

save_manager = SaveManager(modelname='crnn-synthtext', interval=1000000, max_checkpoints=15, plot_interval=5000)

trainer = TrainObjectRecognitionJupyterLogger(LiveGraph(yrange=(0, 2)), CTCLoss(blank=model.blankIndex, zero_infinity=True, reduction='mean'), model, optimizer)
trainer.train_iter(save_manager, 100000000, train_loader)  # , evaluator=VOC2007Evaluator(val_dataset, iteration_interval=10))